<a href="https://colab.research.google.com/github/tessyworld/tessy/blob/main/code_bot_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
import os

import requests
import telegram
from telegram.ext import CommandHandler, Filters, MessageHandler, Updater

# Configura il logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Imposta il token del tuo bot su Telegram
TOKEN = os.environ.get('BOT_TOKEN')

# Crea un'istanza del bot
bot = telegram.Bot(token=TOKEN)

# Funzione per gestire il comando /start
def start(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="Ciao! Sono il bot del meteo. Usa il comando /weather <città> per ottenere le informazioni sul meteo.")

# Funzione per gestire il comando /weather
def weather(update, context):
    city = context.args[0]
    api_key = os.environ.get('OPENWEATHERMAP_API_KEY')
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp']
        description = data['weather'][0]['description']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        text = f"<b>Informazioni meteo per {city}</b>\n\n" \
               f"<b>Temperatura:</b> {temperature}°C\n" \
               f"<b>Descrizione:</b> {description}\n" \
               f"<b>Umidità:</b> {humidity}%\n" \
               f"<b>Velocità del vento:</b> {wind_speed} km/h"

        context.bot.send_message(chat_id=update.message.chat_id, text=text, parse_mode=telegram.ParseMode.HTML)
    else:
        context.bot.send_message(chat_id=update.message.chat_id, text="Spiacenti, non è stato possibile ottenere le informazioni sul meteo.")

# Funzione per gestire i messaggi che non corrispondono a nessun comando
def echo(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="Spiacente, non ho capito il tuo messaggio. Usa il comando /help per vedere la lista dei comandi disponibili.")

# Crea un'istanza di Updater e registra i gestori di comando e di messaggio
updater = Updater(token=TOKEN, use_context=True)

updater.dispatcher.add_handler(CommandHandler('start', start))
updater.dispatcher.add_handler(CommandHandler('weather', weather))
updater.dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Avvia il bot
updater.start_polling()
updater.idle()
